## Выбираем лучшие настройки для тикеров

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from bot.test import TestAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed

# test_configs = [
#     (RunConfig(
#         ticker=t_config.ticker,
#         step_max_cnt=max_shares,
#         step_base_cnt=base_shares,
#         step_lots=1,
# 
#         majority_trade=t_config.majority_trade,
#         pretest_period=pretest_period,
#         pretest_type=t_config.pretest_type,
# 
#         threshold_buy_steps=0,
#         threshold_sell_steps=0,
#         stop_up_p=stop_up_p,
#         stop_down_p=0,
#         start_time=start_time,
# 
#         step_size=t_config.step_size + step_size_shift,
#         step_set_orders_cnt=step_cnt,
#     ))
#     for t_config in [
#         RunConfig.from_repr_string('MTLR+ 2/pre4:0/2 x l1 x 0.4¤'),
#     ]
#     # for max_shares in [3]
#     # for base_shares in [0]
#     # for stop_up_p in [0]
#     # for step_size_shift in [0] # [0, .2, -.2, -.4, .4]
#     # for step_cnt in [2]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
#     # for pretest_period in [5]  # range(3, 7) # todo
# 
#     for max_shares in [3]
#     for start_time in ['07:00', '07:01', '07:05', '07:10', '07:15', '07:30']
#     for base_shares in [0]
#     for stop_up_p in [0]
#     # for stop_up_p in [0, 0.01]
#     for step_size_shift in [0]
#     # for step_size_shift in [0, .2, -.2, -.4, -.6, -.8]
#     for step_cnt in [2]
#     for pretest_period in [t_config.pretest_period]
#     # for pretest_period in range(3, 7)
# ]

# test_configs = [
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| L'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| R'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| LR'),
#     # 
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| L'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| R'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| LR'),
#     
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('RNFT+ 5/pre5:0/2 x l3 x 0.8¤ |u0.01 d0.0|'),
# ]

# test_configs = [
#     RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
#     RunConfig.from_repr_string('MVID+ 4/fan3:0/2 x l1 x 0.2(+x0.2)¤'),
#     RunConfig.from_repr_string('MTLR+ 2/pre4:2/2 x l1 x 1.4¤'),
#     RunConfig.from_repr_string('MTLR+ 2/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 2/fan6:0/2 x l1 x 0.6(+x0.2)¤'),
# ]

test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=t_config.step_max_cnt,
        step_base_cnt=t_config.step_base_cnt,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=t_config.pretest_period,
        pretest_type=t_config.pretest_type,

        stop_up_p=t_config.stop_up_p,

        step_size=t_config.step_size,
        step_size_shift=t_config.step_size_shift,
        step_set_orders_cnt=t_config.step_set_orders_cnt,
        mods=mods,
        # mods=t_config.mods,
    ))
    for t_config in [
        RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
        # RunConfig.from_repr_string('MTLR+ 2/pre4:2/2 x l1 x 1.4¤'),
        # RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
    ]
    for mods in ['']
    # for max_shares in [3]
    # for base_shares in [0]
    # for stop_up_p in [0]
    for step_size_shift in [0]  # [0, .2, .4]
    # for step_cnt in [2]
    # for pretest_period in [t_config.pretest_period]
    # for pretest_period in range(3, 8)
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config, use_cache=True)
    return test_alg.test(
        last_test_date='2024-10-24',
        test_days_num=30,  # todo
        # test_days_num=365,
        shares_count=0,
        

        try_find_best_config=True,
    )

unique_configs = set(test_configs)

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}

    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()


print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)

last_val = ''
for item in sorted_results:
    cur_val = item['config'].ticker
    if cur_val != last_val:
        last_val = cur_val
        print()
    print(item)

Запуск в 17:20
Закончено в 17:21, количество 1, длительность 0:00:12                                               
cache 501 vals, used 480 times

{'exp': 'MVID fan ', 'profit': 1049, 'profit_p': 1.05, 'profit_p_avg': 0.04, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': MVID+ 10/fan7:0/2 x l46 x 0.6(+x0.2)¤ , 'op': 285}


# контрольный результат до доработок


cache 501 vals, used 480 times

{'exp': 'MVID fan ', 'profit': 1049, 'profit_p': 1.05, 'profit_p_avg': 0.04, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': MVID+ 10/fan7:0/2 x l46 x 0.6(+x0.2)¤ , 'op': 285}
